In [ ]:
# BusinessVaultHeader



In [ ]:
# BusinessVaultFooter



In [ ]:
# MartHeader



In [ ]:
# MartFooter



In [ ]:
# RawVaultHeader



In [ ]:
# StageHeader



In [ ]:
# HubTable: CREDITCARD_Hub Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`creditcardid` Integer NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`creditcardid`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,0 AS `creditcardid`
""")



In [ ]:
# HubSourceView: CREDITCARD_Hub Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`creditcardid` AS `creditcardid`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_creditcard_result` AS `s1`
""")



In [ ]:
# HubHashingView: CREDITCARD_Hub Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`creditcardid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`creditcardid` AS `creditcardid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_source`
""")



In [ ]:
# HubResultView: CREDITCARD_Hub Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`creditcardid` AS `creditcardid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# HubDeltaView: CREDITCARD_Hub Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`creditcardid` AS `creditcardid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard` AS `bg_target`
   ON `bg_source`.`hub_hk` = `bg_target`.`hub_hk`
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# SatelliteTable: CREDITCARD_Satellite Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`bg_rowhash` String
    ,`cardtype` String
    ,`cardnumber` String
    ,`expmonth` Integer
    ,`expyear` Timestamp
    ,`modifieddate` Timestamp
    ,`dt` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`bg_validfromtimestamp`
    ,`bg_rowhash`
    ,`cardtype`
    ,`cardnumber`
    ,`expmonth`
    ,`expyear`
    ,`modifieddate`
    ,`dt`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'1900-01-01' AS `bg_validfromtimestamp`
    ,'0' AS `bg_rowhash`
    ,'Unknown' AS `cardtype`
    ,'Unknown' AS `cardnumber`
    ,0 AS `expmonth`
    ,'1900-01-01' AS `expyear`
    ,'1900-01-01' AS `modifieddate`
    ,'Unknown' AS `dt`
""")



In [ ]:
# SatelliteSourceView: CREDITCARD_Satellite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_validfromtimestamp`
    ,`s1`.`creditcardid` AS `fk_creditcard_creditcardid`
    ,`s1`.`cardtype` AS `cardtype`
    ,`s1`.`cardnumber` AS `cardnumber`
    ,`s1`.`expmonth` AS `expmonth`
    ,`s1`.`expyear` AS `expyear`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_creditcard_result` AS `s1`
""")



In [ ]:
# SatelliteHashingView: CREDITCARD_Satellite Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`fk_creditcard_creditcardid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,SHA2(COALESCE(`cardtype`, '_NULL_') || '_' || COALESCE(`cardnumber`, '_NULL_') || '_' || COALESCE(CAST(`expmonth` AS String), '_NULL_') || '_' || COALESCE(CAST(`expyear` AS String), '_NULL_') || '_' || COALESCE(CAST(`modifieddate` AS String), '_NULL_') || '_' || COALESCE(`dt`, '_NULL_'), 256) AS `bg_rowhash`
    ,`fk_creditcard_creditcardid` AS `fk_creditcard_creditcardid`
    ,`cardtype` AS `cardtype`
    ,`cardnumber` AS `cardnumber`
    ,`expmonth` AS `expmonth`
    ,`expyear` AS `expyear`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_source`
""")



In [ ]:
# SatelliteResultView: CREDITCARD_Satellite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_rowhash` AS `bg_rowhash`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `hub_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`cardtype` AS `cardtype`
    ,`cardnumber` AS `cardnumber`
    ,`expmonth` AS `expmonth`
    ,`expyear` AS `expyear`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# SatelliteDeltaView: CREDITCARD_Satellite Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`cardtype` AS `cardtype`
    ,`bg_source`.`cardnumber` AS `cardnumber`
    ,`bg_source`.`expmonth` AS `expmonth`
    ,`bg_source`.`expyear` AS `expyear`
    ,`bg_source`.`modifieddate` AS `modifieddate`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_result` AS `bg_target`
   ON (`bg_source`.`hub_hk` = `bg_target`.`hub_hk`)
  AND (`bg_source`.`bg_rowhash` = `bg_target`.`bg_rowhash`)
  AND (`bg_target`.`bg_validtotimestamp` = '9999-12-31')
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# PitTable: CREDITCARD_PIT Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`pit_hk` String NOT NULL
    ,`creditcard_bg_validfromtimestamp` Timestamp NOT NULL
    ,`creditcard_creditcard_hk` String NOT NULL
)
USING delta
"""
)



In [ ]:
# PitSourceView: CREDITCARD_PIT Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard_source`
AS
SELECT
     `loadset`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`loadset`.`pit_hk` AS `pit_hk`
    ,COALESCE(MAX(`s1`.`bg_validfromtimestamp`), '1900-01-01') AS `creditcard_bg_validfromtimestamp`
    ,COALESCE(MAX(`s1`.`hub_hk`), '0') AS `creditcard_creditcard_hk`
FROM (
    SELECT
         `hub_hk` AS `pit_hk`
        ,`bg_validfromtimestamp`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard`
) AS `loadset`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard_result` AS `s1`
   ON (`s1`.`hub_hk` = `loadset`.`pit_hk`)
  AND (`s1`.`bg_validfromtimestamp` <= `loadset`.`bg_validfromtimestamp`)
GROUP BY
     `loadset`.`bg_validfromtimestamp`
    ,`loadset`.`pit_hk`
""")



In [ ]:
# PitResultView: CREDITCARD_PIT Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `pit_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`pit_hk` AS `pit_hk`
    ,`creditcard_bg_validfromtimestamp` AS `creditcard_bg_validfromtimestamp`
    ,`creditcard_creditcard_hk` AS `creditcard_creditcard_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard`
WHERE `pit_hk` <> '0'
""")



In [ ]:
# PitDeltaView: CREDITCARD_PIT Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard_delta`
AS
SELECT
     `bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`pit_hk` AS `pit_hk`
    ,`bg_source`.`creditcard_bg_validfromtimestamp` AS `creditcard_bg_validfromtimestamp`
    ,`bg_source`.`creditcard_creditcard_hk` AS `creditcard_creditcard_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard_source` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard` AS `bg_target`
   ON (`bg_source`.`pit_hk` = `bg_target`.`pit_hk`)
  AND (`bg_source`.`bg_validfromtimestamp` = `bg_target`.`bg_validfromtimestamp`)
WHERE `bg_target`.`pit_hk` IS NULL
""")



In [ ]:
# CurrentViewResultView: CREDITCARD_Current View Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_current_creditcard`
AS
SELECT
     `s1`.`cardtype` AS `cardtype`
    ,`s1`.`cardnumber` AS `cardnumber`
    ,`s1`.`expmonth` AS `expmonth`
    ,`s1`.`expyear` AS `expyear`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
    ,`s2`.`creditcardid` AS `creditcardid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_creditcard_result` AS `s3`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_creditcard_result` AS `s2`
   ON `s2`.`hub_hk` = `s3`.`pit_hk`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_creditcard` AS `s1`
   ON (`s1`.`hub_hk` = `s3`.`creditcard_creditcard_hk`)
  AND (`s1`.`bg_validfromtimestamp` = `s3`.`creditcard_bg_validfromtimestamp`)
WHERE `s3`.`bg_validtotimestamp` = '9999-12-31'
""")



In [ ]:
# ReferenceTableTable: CURRENCY_Reference Table Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_rowhash` String
    ,`currencycode` String NOT NULL
    ,`name` String
    ,`modifieddate` Timestamp
    ,`dt` String NOT NULL
)
USING delta
"""
)



In [ ]:
# ReferenceTableSourceView: CURRENCY_Reference Table Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`currencycode` AS `currencycode`
    ,`s1`.`name` AS `name`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_currency_result` AS `s1`
""")



In [ ]:
# ReferenceTableHashingView: CURRENCY_Reference Table Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_hashing`
AS
SELECT
     `bg_sourcesystem` AS `bg_sourcesystem`
    ,SHA2(COALESCE(`name`, '_NULL_') || '_' || COALESCE(CAST(`modifieddate` AS String), '_NULL_') || '_' || COALESCE(`dt`, '_NULL_'), 256) AS `bg_rowhash`
    ,`currencycode` AS `currencycode`
    ,`name` AS `name`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_source`
""")



In [ ]:
# ReferenceTableResultView: CURRENCY_Reference Table Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_rowhash` AS `bg_rowhash`
    ,`currencycode` AS `currencycode`
    ,`name` AS `name`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency`
""")



In [ ]:
# ReferenceTableDeltaView: CURRENCY_Reference Table Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_delta`
AS
SELECT
     `bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`currencycode` AS `currencycode`
    ,`bg_source`.`name` AS `name`
    ,`bg_source`.`modifieddate` AS `modifieddate`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_referencetable_currency_result` AS `bg_target`
   ON (`bg_source`.`currencycode` = `bg_target`.`currencycode`)
WHERE (`bg_target`.`bg_loadtimestamp` IS NULL)
   OR (`bg_source`.`bg_rowhash` <> `bg_target`.`bg_rowhash`)
""")



In [ ]:
# SatelliteTable: CUSTOMER_Satellite Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`bg_rowhash` String
    ,`customerid` Integer
    ,`personid` Integer
    ,`storeid` Integer
    ,`territoryid` Integer
    ,`rowguid` String
    ,`modifieddate` Timestamp
    ,`dt` String
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`bg_validfromtimestamp`
    ,`bg_rowhash`
    ,`customerid`
    ,`personid`
    ,`storeid`
    ,`territoryid`
    ,`rowguid`
    ,`modifieddate`
    ,`dt`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'1900-01-01' AS `bg_validfromtimestamp`
    ,'0' AS `bg_rowhash`
    ,0 AS `customerid`
    ,0 AS `personid`
    ,0 AS `storeid`
    ,0 AS `territoryid`
    ,'Unknown' AS `rowguid`
    ,'1900-01-01' AS `modifieddate`
    ,'Unknown' AS `dt`
""")



In [ ]:
# SatelliteSourceView: CUSTOMER_Satellite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_validfromtimestamp`
    ,`s1`.`accountnumber` AS `fk_customer_accountnumber`
    ,`s1`.`customerid` AS `customerid`
    ,`s1`.`personid` AS `personid`
    ,`s1`.`storeid` AS `storeid`
    ,`s1`.`territoryid` AS `territoryid`
    ,`s1`.`rowguid` AS `rowguid`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_customer_result` AS `s1`
""")



In [ ]:
# SatelliteHashingView: CUSTOMER_Satellite Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_hashing`
AS
SELECT
     SHA2(COALESCE(`fk_customer_accountnumber`, '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,SHA2(COALESCE(CAST(`customerid` AS String), '_NULL_') || '_' || COALESCE(CAST(`personid` AS String), '_NULL_') || '_' || COALESCE(CAST(`storeid` AS String), '_NULL_') || '_' || COALESCE(CAST(`territoryid` AS String), '_NULL_') || '_' || COALESCE(`rowguid`, '_NULL_') || '_' || COALESCE(CAST(`modifieddate` AS String), '_NULL_') || '_' || COALESCE(`dt`, '_NULL_'), 256) AS `bg_rowhash`
    ,`fk_customer_accountnumber` AS `fk_customer_accountnumber`
    ,`customerid` AS `customerid`
    ,`personid` AS `personid`
    ,`storeid` AS `storeid`
    ,`territoryid` AS `territoryid`
    ,`rowguid` AS `rowguid`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_source`
""")



In [ ]:
# SatelliteResultView: CUSTOMER_Satellite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_rowhash` AS `bg_rowhash`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `hub_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`customerid` AS `customerid`
    ,`personid` AS `personid`
    ,`storeid` AS `storeid`
    ,`territoryid` AS `territoryid`
    ,`rowguid` AS `rowguid`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# SatelliteDeltaView: CUSTOMER_Satellite Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`customerid` AS `customerid`
    ,`bg_source`.`personid` AS `personid`
    ,`bg_source`.`storeid` AS `storeid`
    ,`bg_source`.`territoryid` AS `territoryid`
    ,`bg_source`.`rowguid` AS `rowguid`
    ,`bg_source`.`modifieddate` AS `modifieddate`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_result` AS `bg_target`
   ON (`bg_source`.`hub_hk` = `bg_target`.`hub_hk`)
  AND (`bg_source`.`bg_rowhash` = `bg_target`.`bg_rowhash`)
  AND (`bg_target`.`bg_validtotimestamp` = '9999-12-31')
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# HubTable: CUSTOMER_Hub Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`accountnumber` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`accountnumber`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'Unknown' AS `accountnumber`
""")



In [ ]:
# HubSourceView: CUSTOMER_Hub Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`accountnumber` AS `accountnumber`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_customer_result` AS `s1`
UNION
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`accountnumber` AS `accountnumber`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_customercrm_result` AS `s1`
""")



In [ ]:
# HubDeduplicationView: CUSTOMER_Hub Deduplication View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_deduplication`
AS
SELECT
     `bg_partition`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_partition`.`accountnumber` AS `accountnumber`
FROM (
    SELECT
         `bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
        ,`bg_source`.`accountnumber` AS `accountnumber`
        ,ROW_NUMBER() OVER (PARTITION BY `bg_source`.`accountnumber` ORDER BY `bg_source`.`accountnumber` ASC) AS `bg_deduplicationpartitionorder`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_source` AS `bg_source`
) AS `bg_partition`
WHERE `bg_partition`.`bg_deduplicationpartitionorder` = 1
""")



In [ ]:
# HubHashingView: CUSTOMER_Hub Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_hashing`
AS
SELECT
     SHA2(COALESCE(`accountnumber`, '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`accountnumber` AS `accountnumber`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_deduplication`
""")



In [ ]:
# HubResultView: CUSTOMER_Hub Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`accountnumber` AS `accountnumber`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# HubDeltaView: CUSTOMER_Hub Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`accountnumber` AS `accountnumber`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer` AS `bg_target`
   ON `bg_source`.`hub_hk` = `bg_target`.`hub_hk`
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# SatelliteTable: CUSTOMERCRM_Satellite Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`bg_rowhash` String
    ,`marital_status` String
    ,`gender` String
    ,`number_of_children` Integer
    ,`dt` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`bg_validfromtimestamp`
    ,`bg_rowhash`
    ,`marital_status`
    ,`gender`
    ,`number_of_children`
    ,`dt`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'1900-01-01' AS `bg_validfromtimestamp`
    ,'0' AS `bg_rowhash`
    ,'Unknown' AS `marital_status`
    ,'Unknown' AS `gender`
    ,0 AS `number_of_children`
    ,'Unknown' AS `dt`
""")



In [ ]:
# SatelliteSourceView: CUSTOMERCRM_Satellite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_validfromtimestamp`
    ,`s1`.`accountnumber` AS `fk_customer_accountnumber`
    ,`s1`.`marital_status` AS `marital_status`
    ,`s1`.`gender` AS `gender`
    ,`s1`.`number_of_children` AS `number_of_children`
    ,`s1`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_customercrm_result` AS `s1`
""")



In [ ]:
# SatelliteHashingView: CUSTOMERCRM_Satellite Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_hashing`
AS
SELECT
     SHA2(COALESCE(`fk_customer_accountnumber`, '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,SHA2(COALESCE(`marital_status`, '_NULL_') || '_' || COALESCE(`gender`, '_NULL_') || '_' || COALESCE(CAST(`number_of_children` AS String), '_NULL_') || '_' || COALESCE(`dt`, '_NULL_'), 256) AS `bg_rowhash`
    ,`fk_customer_accountnumber` AS `fk_customer_accountnumber`
    ,`marital_status` AS `marital_status`
    ,`gender` AS `gender`
    ,`number_of_children` AS `number_of_children`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_source`
""")



In [ ]:
# SatelliteResultView: CUSTOMERCRM_Satellite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_rowhash` AS `bg_rowhash`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `hub_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`marital_status` AS `marital_status`
    ,`gender` AS `gender`
    ,`number_of_children` AS `number_of_children`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# SatelliteDeltaView: CUSTOMERCRM_Satellite Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`marital_status` AS `marital_status`
    ,`bg_source`.`gender` AS `gender`
    ,`bg_source`.`number_of_children` AS `number_of_children`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_result` AS `bg_target`
   ON (`bg_source`.`hub_hk` = `bg_target`.`hub_hk`)
  AND (`bg_source`.`bg_rowhash` = `bg_target`.`bg_rowhash`)
  AND (`bg_target`.`bg_validtotimestamp` = '9999-12-31')
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# PitTable: CUSTOMER_PIT Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`pit_hk` String NOT NULL
    ,`customer_bg_validfromtimestamp` Timestamp NOT NULL
    ,`customer_customer_hk` String NOT NULL
    ,`customercrm_bg_validfromtimestamp` Timestamp NOT NULL
    ,`customercrm_customercrm_hk` String NOT NULL
)
USING delta
"""
)



In [ ]:
# PitSourceView: CUSTOMER_PIT Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer_source`
AS
SELECT
     `loadset`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`loadset`.`pit_hk` AS `pit_hk`
    ,COALESCE(MAX(`s1`.`bg_validfromtimestamp`), '1900-01-01') AS `customer_bg_validfromtimestamp`
    ,COALESCE(MAX(`s1`.`hub_hk`), '0') AS `customer_customer_hk`
    ,COALESCE(MAX(`s2`.`bg_validfromtimestamp`), '1900-01-01') AS `customercrm_bg_validfromtimestamp`
    ,COALESCE(MAX(`s2`.`hub_hk`), '0') AS `customercrm_customercrm_hk`
FROM (
    SELECT
         `hub_hk` AS `pit_hk`
        ,`bg_validfromtimestamp`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer`
    UNION
    SELECT
         `hub_hk` AS `pit_hk`
        ,`bg_validfromtimestamp`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm`
) AS `loadset`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer_result` AS `s1`
   ON (`s1`.`hub_hk` = `loadset`.`pit_hk`)
  AND (`s1`.`bg_validfromtimestamp` <= `loadset`.`bg_validfromtimestamp`)
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm_result` AS `s2`
   ON (`s2`.`hub_hk` = `loadset`.`pit_hk`)
  AND (`s2`.`bg_validfromtimestamp` <= `loadset`.`bg_validfromtimestamp`)
GROUP BY
     `loadset`.`bg_validfromtimestamp`
    ,`loadset`.`pit_hk`
""")



In [ ]:
# PitResultView: CUSTOMER_PIT Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `pit_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`pit_hk` AS `pit_hk`
    ,`customer_bg_validfromtimestamp` AS `customer_bg_validfromtimestamp`
    ,`customer_customer_hk` AS `customer_customer_hk`
    ,`customercrm_bg_validfromtimestamp` AS `customercrm_bg_validfromtimestamp`
    ,`customercrm_customercrm_hk` AS `customercrm_customercrm_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer`
WHERE `pit_hk` <> '0'
""")



In [ ]:
# PitDeltaView: CUSTOMER_PIT Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer_delta`
AS
SELECT
     `bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`pit_hk` AS `pit_hk`
    ,`bg_source`.`customer_bg_validfromtimestamp` AS `customer_bg_validfromtimestamp`
    ,`bg_source`.`customer_customer_hk` AS `customer_customer_hk`
    ,`bg_source`.`customercrm_bg_validfromtimestamp` AS `customercrm_bg_validfromtimestamp`
    ,`bg_source`.`customercrm_customercrm_hk` AS `customercrm_customercrm_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer_source` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer` AS `bg_target`
   ON (`bg_source`.`pit_hk` = `bg_target`.`pit_hk`)
  AND (`bg_source`.`bg_validfromtimestamp` = `bg_target`.`bg_validfromtimestamp`)
WHERE `bg_target`.`pit_hk` IS NULL
""")



In [ ]:
# CurrentViewResultView: CUSTOMER_Current View Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_current_customer`
AS
SELECT
     `s1`.`marital_status` AS `marital_status`
    ,`s1`.`gender` AS `gender`
    ,`s1`.`number_of_children` AS `number_of_children`
    ,`s1`.`dt` AS `dt`
    ,`s2`.`customerid` AS `customerid`
    ,`s2`.`personid` AS `personid`
    ,`s2`.`storeid` AS `storeid`
    ,`s2`.`territoryid` AS `territoryid`
    ,`s2`.`rowguid` AS `rowguid`
    ,`s2`.`modifieddate` AS `modifieddate`
    ,`s4`.`accountnumber` AS `accountnumber`
    ,`s2`.`hub_hk` AS `hub_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_customer_result` AS `s3`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_customer_result` AS `s4`
   ON `s4`.`hub_hk` = `s3`.`pit_hk`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customercrm` AS `s1`
   ON (`s1`.`hub_hk` = `s3`.`customercrm_customercrm_hk`)
  AND (`s1`.`bg_validfromtimestamp` = `s3`.`customercrm_bg_validfromtimestamp`)
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_customer` AS `s2`
   ON (`s2`.`hub_hk` = `s3`.`customer_customer_hk`)
  AND (`s2`.`bg_validfromtimestamp` = `s3`.`customer_bg_validfromtimestamp`)
WHERE `s3`.`bg_validtotimestamp` = '9999-12-31'
""")



In [ ]:
# HubTable: ORDERDETAIL_Hub Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`salesorderid` Integer NOT NULL
    ,`salesorderdetailid` Integer NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`salesorderid`
    ,`salesorderdetailid`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,0 AS `salesorderid`
    ,0 AS `salesorderdetailid`
""")



In [ ]:
# HubSourceView: ORDERDETAIL_Hub Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`salesorderid` AS `salesorderid`
    ,`s1`.`salesorderdetailid` AS `salesorderdetailid`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderdetail_result` AS `s1`
""")



In [ ]:
# HubHashingView: ORDERDETAIL_Hub Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`salesorderid` AS String), '_NULL_') || '_' || COALESCE(CAST(`salesorderdetailid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`salesorderid` AS `salesorderid`
    ,`salesorderdetailid` AS `salesorderdetailid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_source`
""")



In [ ]:
# HubResultView: ORDERDETAIL_Hub Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`salesorderid` AS `salesorderid`
    ,`salesorderdetailid` AS `salesorderdetailid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# HubDeltaView: ORDERDETAIL_Hub Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`salesorderid` AS `salesorderid`
    ,`bg_source`.`salesorderdetailid` AS `salesorderdetailid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail` AS `bg_target`
   ON `bg_source`.`hub_hk` = `bg_target`.`hub_hk`
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# SatelliteTable: ORDERDETAIL_Satellite Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`bg_rowhash` String
    ,`carriertrackingnumber` String
    ,`orderqty` Integer
    ,`productid` Integer
    ,`specialofferid` Integer
    ,`unitprice` Decimal(19,4)
    ,`unitpricediscount` Decimal(19,4)
    ,`linetotal` Decimal(38,6)
    ,`rowguid` String
    ,`modifieddate` Timestamp
    ,`dt` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`bg_validfromtimestamp`
    ,`bg_rowhash`
    ,`carriertrackingnumber`
    ,`orderqty`
    ,`productid`
    ,`specialofferid`
    ,`unitprice`
    ,`unitpricediscount`
    ,`linetotal`
    ,`rowguid`
    ,`modifieddate`
    ,`dt`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'1900-01-01' AS `bg_validfromtimestamp`
    ,'0' AS `bg_rowhash`
    ,'Unknown' AS `carriertrackingnumber`
    ,0 AS `orderqty`
    ,0 AS `productid`
    ,0 AS `specialofferid`
    ,0 AS `unitprice`
    ,0 AS `unitpricediscount`
    ,0 AS `linetotal`
    ,'Unknown' AS `rowguid`
    ,'1900-01-01' AS `modifieddate`
    ,'Unknown' AS `dt`
""")



In [ ]:
# SatelliteSourceView: ORDERDETAIL_Satellite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_validfromtimestamp`
    ,`s1`.`salesorderid` AS `fk_orderdetail_salesorderid`
    ,`s1`.`salesorderdetailid` AS `fk_orderdetail_salesorderdetailid`
    ,`s1`.`carriertrackingnumber` AS `carriertrackingnumber`
    ,`s1`.`orderqty` AS `orderqty`
    ,`s1`.`productid` AS `productid`
    ,`s1`.`specialofferid` AS `specialofferid`
    ,`s1`.`unitprice` AS `unitprice`
    ,`s1`.`unitpricediscount` AS `unitpricediscount`
    ,`s1`.`linetotal` AS `linetotal`
    ,`s1`.`rowguid` AS `rowguid`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderdetail_result` AS `s1`
""")



In [ ]:
# SatelliteHashingView: ORDERDETAIL_Satellite Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`fk_orderdetail_salesorderid` AS String), '_NULL_') || '_' || COALESCE(CAST(`fk_orderdetail_salesorderdetailid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,SHA2(COALESCE(`carriertrackingnumber`, '_NULL_') || '_' || COALESCE(CAST(`orderqty` AS String), '_NULL_') || '_' || COALESCE(CAST(`productid` AS String), '_NULL_') || '_' || COALESCE(CAST(`specialofferid` AS String), '_NULL_') || '_' || COALESCE(CAST(`unitprice` AS String), '_NULL_') || '_' || COALESCE(CAST(`unitpricediscount` AS String), '_NULL_') || '_' || COALESCE(CAST(`linetotal` AS String), '_NULL_') || '_' || COALESCE(`rowguid`, '_NULL_') || '_' || COALESCE(CAST(`modifieddate` AS String), '_NULL_') || '_' || COALESCE(`dt`, '_NULL_'), 256) AS `bg_rowhash`
    ,`fk_orderdetail_salesorderid` AS `fk_orderdetail_salesorderid`
    ,`fk_orderdetail_salesorderdetailid` AS `fk_orderdetail_salesorderdetailid`
    ,`carriertrackingnumber` AS `carriertrackingnumber`
    ,`orderqty` AS `orderqty`
    ,`productid` AS `productid`
    ,`specialofferid` AS `specialofferid`
    ,`unitprice` AS `unitprice`
    ,`unitpricediscount` AS `unitpricediscount`
    ,`linetotal` AS `linetotal`
    ,`rowguid` AS `rowguid`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_source`
""")



In [ ]:
# SatelliteResultView: ORDERDETAIL_Satellite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_rowhash` AS `bg_rowhash`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `hub_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`carriertrackingnumber` AS `carriertrackingnumber`
    ,`orderqty` AS `orderqty`
    ,`productid` AS `productid`
    ,`specialofferid` AS `specialofferid`
    ,`unitprice` AS `unitprice`
    ,`unitpricediscount` AS `unitpricediscount`
    ,`linetotal` AS `linetotal`
    ,`rowguid` AS `rowguid`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# SatelliteDeltaView: ORDERDETAIL_Satellite Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`carriertrackingnumber` AS `carriertrackingnumber`
    ,`bg_source`.`orderqty` AS `orderqty`
    ,`bg_source`.`productid` AS `productid`
    ,`bg_source`.`specialofferid` AS `specialofferid`
    ,`bg_source`.`unitprice` AS `unitprice`
    ,`bg_source`.`unitpricediscount` AS `unitpricediscount`
    ,`bg_source`.`linetotal` AS `linetotal`
    ,`bg_source`.`rowguid` AS `rowguid`
    ,`bg_source`.`modifieddate` AS `modifieddate`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_result` AS `bg_target`
   ON (`bg_source`.`hub_hk` = `bg_target`.`hub_hk`)
  AND (`bg_source`.`bg_rowhash` = `bg_target`.`bg_rowhash`)
  AND (`bg_target`.`bg_validtotimestamp` = '9999-12-31')
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# PitTable: ORDERDETAIL_PIT Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`pit_hk` String NOT NULL
    ,`orderdetail_bg_validfromtimestamp` Timestamp NOT NULL
    ,`orderdetail_orderdetail_hk` String NOT NULL
)
USING delta
"""
)



In [ ]:
# PitSourceView: ORDERDETAIL_PIT Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail_source`
AS
SELECT
     `loadset`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`loadset`.`pit_hk` AS `pit_hk`
    ,COALESCE(MAX(`s1`.`bg_validfromtimestamp`), '1900-01-01') AS `orderdetail_bg_validfromtimestamp`
    ,COALESCE(MAX(`s1`.`hub_hk`), '0') AS `orderdetail_orderdetail_hk`
FROM (
    SELECT
         `hub_hk` AS `pit_hk`
        ,`bg_validfromtimestamp`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail`
) AS `loadset`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail_result` AS `s1`
   ON (`s1`.`hub_hk` = `loadset`.`pit_hk`)
  AND (`s1`.`bg_validfromtimestamp` <= `loadset`.`bg_validfromtimestamp`)
GROUP BY
     `loadset`.`bg_validfromtimestamp`
    ,`loadset`.`pit_hk`
""")



In [ ]:
# PitResultView: ORDERDETAIL_PIT Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `pit_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`pit_hk` AS `pit_hk`
    ,`orderdetail_bg_validfromtimestamp` AS `orderdetail_bg_validfromtimestamp`
    ,`orderdetail_orderdetail_hk` AS `orderdetail_orderdetail_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail`
WHERE `pit_hk` <> '0'
""")



In [ ]:
# PitDeltaView: ORDERDETAIL_PIT Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail_delta`
AS
SELECT
     `bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`pit_hk` AS `pit_hk`
    ,`bg_source`.`orderdetail_bg_validfromtimestamp` AS `orderdetail_bg_validfromtimestamp`
    ,`bg_source`.`orderdetail_orderdetail_hk` AS `orderdetail_orderdetail_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail_source` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail` AS `bg_target`
   ON (`bg_source`.`pit_hk` = `bg_target`.`pit_hk`)
  AND (`bg_source`.`bg_validfromtimestamp` = `bg_target`.`bg_validfromtimestamp`)
WHERE `bg_target`.`pit_hk` IS NULL
""")



In [ ]:
# CurrentViewResultView: ORDERDETAIL_Current View Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_current_orderdetail`
AS
SELECT
     `s1`.`carriertrackingnumber` AS `carriertrackingnumber`
    ,`s1`.`orderqty` AS `orderqty`
    ,`s1`.`productid` AS `productid`
    ,`s1`.`specialofferid` AS `specialofferid`
    ,`s1`.`unitprice` AS `unitprice`
    ,`s1`.`unitpricediscount` AS `unitpricediscount`
    ,`s1`.`linetotal` AS `linetotal`
    ,`s1`.`rowguid` AS `rowguid`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
    ,`s2`.`salesorderid` AS `salesorderid`
    ,`s2`.`salesorderdetailid` AS `salesorderdetailid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_orderdetail_result` AS `s3`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_orderdetail_result` AS `s2`
   ON `s2`.`hub_hk` = `s3`.`pit_hk`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_orderdetail` AS `s1`
   ON (`s1`.`hub_hk` = `s3`.`orderdetail_orderdetail_hk`)
  AND (`s1`.`bg_validfromtimestamp` = `s3`.`orderdetail_bg_validfromtimestamp`)
WHERE `s3`.`bg_validtotimestamp` = '9999-12-31'
""")



In [ ]:
# LinkTable: CUSTOMER_ORDER_Link Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`link_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`fk_customer_accountnumber` String NOT NULL
    ,`customer_customer_hk` String NOT NULL
    ,`fk_order_salesorderid` Integer NOT NULL
    ,`order_order_hk` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order` (
     `bg_loadtimestamp`
    ,`link_hk`
    ,`bg_sourcesystem`
    ,`fk_customer_accountnumber`
    ,`customer_customer_hk`
    ,`fk_order_salesorderid`
    ,`order_order_hk`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `link_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'Unknown' AS `fk_customer_accountnumber`
    ,'0' AS `customer_customer_hk`
    ,0 AS `fk_order_salesorderid`
    ,'0' AS `order_order_hk`
""")



In [ ]:
# LinkSourceView: CUSTOMER_ORDER_Link Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`accountnumber` AS `fk_customer_accountnumber`
    ,`s2`.`salesorderid` AS `fk_order_salesorderid`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_customer_result` AS `s1`
JOIN `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderheader_result` AS `s2`
   ON `s1`.`customerid` = `s2`.`customerid`
""")



In [ ]:
# LinkHashingView: CUSTOMER_ORDER_Link Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order_hashing`
AS
SELECT
     SHA2(COALESCE(`fk_customer_accountnumber`, '_NULL_') || '_' || COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `link_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`fk_customer_accountnumber` AS `fk_customer_accountnumber`
    ,SHA2(COALESCE(`fk_customer_accountnumber`, '_NULL_'), 256) AS `customer_customer_hk`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,SHA2(COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `order_order_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order_source`
""")



In [ ]:
# LinkresultView: CUSTOMER_ORDER_Link Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`link_hk` AS `link_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`fk_customer_accountnumber` AS `fk_customer_accountnumber`
    ,`customer_customer_hk` AS `customer_customer_hk`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`order_order_hk` AS `order_order_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order`
WHERE `link_hk` <> '0'
""")



In [ ]:
# LinkDeltaView: CUSTOMER_ORDER_Link Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order_delta`
AS
SELECT
     `bg_source`.`link_hk` AS `link_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`fk_customer_accountnumber` AS `fk_customer_accountnumber`
    ,`bg_source`.`customer_customer_hk` AS `customer_customer_hk`
    ,`bg_source`.`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`bg_source`.`order_order_hk` AS `order_order_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_customer_order` AS `bg_target`
   ON `bg_source`.`link_hk` = `bg_target`.`link_hk`
WHERE `bg_target`.`link_hk` IS NULL
""")



In [ ]:
# HubTable: ORDER_Hub Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`salesorderid` Integer NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`salesorderid`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,0 AS `salesorderid`
""")



In [ ]:
# HubSourceView: ORDER_Hub Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s1`.`salesorderid` AS `salesorderid`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderheader_result` AS `s1`
""")



In [ ]:
# HubHashingView: ORDER_Hub Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`salesorderid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`salesorderid` AS `salesorderid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_source`
""")



In [ ]:
# HubResultView: ORDER_Hub Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`salesorderid` AS `salesorderid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# HubDeltaView: ORDER_Hub Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`salesorderid` AS `salesorderid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order` AS `bg_target`
   ON `bg_source`.`hub_hk` = `bg_target`.`hub_hk`
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# SatelliteTable: ORDER_Satellite Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`bg_rowhash` String
    ,`revisionnumber` Integer
    ,`orderdate` Timestamp
    ,`duedate` Timestamp
    ,`shipdate` Timestamp
    ,`status` Integer
    ,`onlineorderflag` Integer
    ,`salesordernumber` String
    ,`purchaseordernumber` String
    ,`accountnumber` String
    ,`customerid` Integer
    ,`salespersonid` Integer
    ,`territoryid` Integer
    ,`billtoaddressid` Integer
    ,`shiptoaddressid` Integer
    ,`shipmethodid` Integer
    ,`creditcardid` Integer
    ,`creditcardapprovalcode` String
    ,`subtotal` Decimal(19,4)
    ,`currencyrateid` Integer
    ,`taxamt` Decimal(19,4)
    ,`freight` Decimal(19,4)
    ,`totaldue` Decimal(19,4)
    ,`comment` String
    ,`rowguid` String
    ,`modifieddate` Timestamp
    ,`dt` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`bg_validfromtimestamp`
    ,`bg_rowhash`
    ,`revisionnumber`
    ,`orderdate`
    ,`duedate`
    ,`shipdate`
    ,`status`
    ,`onlineorderflag`
    ,`salesordernumber`
    ,`purchaseordernumber`
    ,`accountnumber`
    ,`customerid`
    ,`salespersonid`
    ,`territoryid`
    ,`billtoaddressid`
    ,`shiptoaddressid`
    ,`shipmethodid`
    ,`creditcardid`
    ,`creditcardapprovalcode`
    ,`subtotal`
    ,`currencyrateid`
    ,`taxamt`
    ,`freight`
    ,`totaldue`
    ,`comment`
    ,`rowguid`
    ,`modifieddate`
    ,`dt`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'1900-01-01' AS `bg_validfromtimestamp`
    ,'0' AS `bg_rowhash`
    ,0 AS `revisionnumber`
    ,'1900-01-01' AS `orderdate`
    ,'1900-01-01' AS `duedate`
    ,'1900-01-01' AS `shipdate`
    ,0 AS `status`
    ,0 AS `onlineorderflag`
    ,'Unknown' AS `salesordernumber`
    ,'Unknown' AS `purchaseordernumber`
    ,'Unknown' AS `accountnumber`
    ,0 AS `customerid`
    ,0 AS `salespersonid`
    ,0 AS `territoryid`
    ,0 AS `billtoaddressid`
    ,0 AS `shiptoaddressid`
    ,0 AS `shipmethodid`
    ,0 AS `creditcardid`
    ,'Unknown' AS `creditcardapprovalcode`
    ,0 AS `subtotal`
    ,0 AS `currencyrateid`
    ,0 AS `taxamt`
    ,0 AS `freight`
    ,0 AS `totaldue`
    ,'Unknown' AS `comment`
    ,'Unknown' AS `rowguid`
    ,'1900-01-01' AS `modifieddate`
    ,'Unknown' AS `dt`
""")



In [ ]:
# SatelliteSourceView: ORDER_Satellite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_validfromtimestamp`
    ,`s1`.`salesorderid` AS `fk_order_salesorderid`
    ,`s1`.`revisionnumber` AS `revisionnumber`
    ,`s1`.`orderdate` AS `orderdate`
    ,`s1`.`duedate` AS `duedate`
    ,`s1`.`shipdate` AS `shipdate`
    ,`s1`.`status` AS `status`
    ,`s1`.`onlineorderflag` AS `onlineorderflag`
    ,`s1`.`salesordernumber` AS `salesordernumber`
    ,`s1`.`purchaseordernumber` AS `purchaseordernumber`
    ,`s1`.`accountnumber` AS `accountnumber`
    ,`s1`.`customerid` AS `customerid`
    ,`s1`.`salespersonid` AS `salespersonid`
    ,`s1`.`territoryid` AS `territoryid`
    ,`s1`.`billtoaddressid` AS `billtoaddressid`
    ,`s1`.`shiptoaddressid` AS `shiptoaddressid`
    ,`s1`.`shipmethodid` AS `shipmethodid`
    ,`s1`.`creditcardid` AS `creditcardid`
    ,`s1`.`creditcardapprovalcode` AS `creditcardapprovalcode`
    ,`s1`.`subtotal` AS `subtotal`
    ,`s1`.`currencyrateid` AS `currencyrateid`
    ,`s1`.`taxamt` AS `taxamt`
    ,`s1`.`freight` AS `freight`
    ,`s1`.`totaldue` AS `totaldue`
    ,`s1`.`comment` AS `comment`
    ,`s1`.`rowguid` AS `rowguid`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderheader_result` AS `s1`
""")



In [ ]:
# SatelliteHashingView: ORDER_Satellite Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,SHA2(COALESCE(CAST(`revisionnumber` AS String), '_NULL_') || '_' || COALESCE(CAST(`orderdate` AS String), '_NULL_') || '_' || COALESCE(CAST(`duedate` AS String), '_NULL_') || '_' || COALESCE(CAST(`shipdate` AS String), '_NULL_') || '_' || COALESCE(CAST(`status` AS String), '_NULL_') || '_' || COALESCE(CAST(`onlineorderflag` AS String), '_NULL_') || '_' || COALESCE(`salesordernumber`, '_NULL_') || '_' || COALESCE(`purchaseordernumber`, '_NULL_') || '_' || COALESCE(`accountnumber`, '_NULL_') || '_' || COALESCE(CAST(`customerid` AS String), '_NULL_') || '_' || COALESCE(CAST(`salespersonid` AS String), '_NULL_') || '_' || COALESCE(CAST(`territoryid` AS String), '_NULL_') || '_' || COALESCE(CAST(`billtoaddressid` AS String), '_NULL_') || '_' || COALESCE(CAST(`shiptoaddressid` AS String), '_NULL_') || '_' || COALESCE(CAST(`shipmethodid` AS String), '_NULL_') || '_' || COALESCE(CAST(`creditcardid` AS String), '_NULL_') || '_' || COALESCE(`creditcardapprovalcode`, '_NULL_') || '_' || COALESCE(CAST(`subtotal` AS String), '_NULL_') || '_' || COALESCE(CAST(`currencyrateid` AS String), '_NULL_') || '_' || COALESCE(CAST(`taxamt` AS String), '_NULL_') || '_' || COALESCE(CAST(`freight` AS String), '_NULL_') || '_' || COALESCE(CAST(`totaldue` AS String), '_NULL_') || '_' || COALESCE(`comment`, '_NULL_') || '_' || COALESCE(`rowguid`, '_NULL_') || '_' || COALESCE(CAST(`modifieddate` AS String), '_NULL_') || '_' || COALESCE(`dt`, '_NULL_'), 256) AS `bg_rowhash`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`revisionnumber` AS `revisionnumber`
    ,`orderdate` AS `orderdate`
    ,`duedate` AS `duedate`
    ,`shipdate` AS `shipdate`
    ,`status` AS `status`
    ,`onlineorderflag` AS `onlineorderflag`
    ,`salesordernumber` AS `salesordernumber`
    ,`purchaseordernumber` AS `purchaseordernumber`
    ,`accountnumber` AS `accountnumber`
    ,`customerid` AS `customerid`
    ,`salespersonid` AS `salespersonid`
    ,`territoryid` AS `territoryid`
    ,`billtoaddressid` AS `billtoaddressid`
    ,`shiptoaddressid` AS `shiptoaddressid`
    ,`shipmethodid` AS `shipmethodid`
    ,`creditcardid` AS `creditcardid`
    ,`creditcardapprovalcode` AS `creditcardapprovalcode`
    ,`subtotal` AS `subtotal`
    ,`currencyrateid` AS `currencyrateid`
    ,`taxamt` AS `taxamt`
    ,`freight` AS `freight`
    ,`totaldue` AS `totaldue`
    ,`comment` AS `comment`
    ,`rowguid` AS `rowguid`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_source`
""")



In [ ]:
# SatelliteResultView: ORDER_Satellite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_rowhash` AS `bg_rowhash`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `hub_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`revisionnumber` AS `revisionnumber`
    ,`orderdate` AS `orderdate`
    ,`duedate` AS `duedate`
    ,`shipdate` AS `shipdate`
    ,`status` AS `status`
    ,`onlineorderflag` AS `onlineorderflag`
    ,`salesordernumber` AS `salesordernumber`
    ,`purchaseordernumber` AS `purchaseordernumber`
    ,`accountnumber` AS `accountnumber`
    ,`customerid` AS `customerid`
    ,`salespersonid` AS `salespersonid`
    ,`territoryid` AS `territoryid`
    ,`billtoaddressid` AS `billtoaddressid`
    ,`shiptoaddressid` AS `shiptoaddressid`
    ,`shipmethodid` AS `shipmethodid`
    ,`creditcardid` AS `creditcardid`
    ,`creditcardapprovalcode` AS `creditcardapprovalcode`
    ,`subtotal` AS `subtotal`
    ,`currencyrateid` AS `currencyrateid`
    ,`taxamt` AS `taxamt`
    ,`freight` AS `freight`
    ,`totaldue` AS `totaldue`
    ,`comment` AS `comment`
    ,`rowguid` AS `rowguid`
    ,`modifieddate` AS `modifieddate`
    ,`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# SatelliteDeltaView: ORDER_Satellite Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`revisionnumber` AS `revisionnumber`
    ,`bg_source`.`orderdate` AS `orderdate`
    ,`bg_source`.`duedate` AS `duedate`
    ,`bg_source`.`shipdate` AS `shipdate`
    ,`bg_source`.`status` AS `status`
    ,`bg_source`.`onlineorderflag` AS `onlineorderflag`
    ,`bg_source`.`salesordernumber` AS `salesordernumber`
    ,`bg_source`.`purchaseordernumber` AS `purchaseordernumber`
    ,`bg_source`.`accountnumber` AS `accountnumber`
    ,`bg_source`.`customerid` AS `customerid`
    ,`bg_source`.`salespersonid` AS `salespersonid`
    ,`bg_source`.`territoryid` AS `territoryid`
    ,`bg_source`.`billtoaddressid` AS `billtoaddressid`
    ,`bg_source`.`shiptoaddressid` AS `shiptoaddressid`
    ,`bg_source`.`shipmethodid` AS `shipmethodid`
    ,`bg_source`.`creditcardid` AS `creditcardid`
    ,`bg_source`.`creditcardapprovalcode` AS `creditcardapprovalcode`
    ,`bg_source`.`subtotal` AS `subtotal`
    ,`bg_source`.`currencyrateid` AS `currencyrateid`
    ,`bg_source`.`taxamt` AS `taxamt`
    ,`bg_source`.`freight` AS `freight`
    ,`bg_source`.`totaldue` AS `totaldue`
    ,`bg_source`.`comment` AS `comment`
    ,`bg_source`.`rowguid` AS `rowguid`
    ,`bg_source`.`modifieddate` AS `modifieddate`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_result` AS `bg_target`
   ON (`bg_source`.`hub_hk` = `bg_target`.`hub_hk`)
  AND (`bg_source`.`bg_rowhash` = `bg_target`.`bg_rowhash`)
  AND (`bg_target`.`bg_validtotimestamp` = '9999-12-31')
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# LinkTable: ORDER_CREDITCARD_Link Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`link_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`fk_creditcard_creditcardid` Integer NOT NULL
    ,`creditcard_creditcard_hk` String NOT NULL
    ,`fk_order_salesorderid` Integer NOT NULL
    ,`order_order_hk` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard` (
     `bg_loadtimestamp`
    ,`link_hk`
    ,`bg_sourcesystem`
    ,`fk_creditcard_creditcardid`
    ,`creditcard_creditcard_hk`
    ,`fk_order_salesorderid`
    ,`order_order_hk`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `link_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,0 AS `fk_creditcard_creditcardid`
    ,'0' AS `creditcard_creditcard_hk`
    ,0 AS `fk_order_salesorderid`
    ,'0' AS `order_order_hk`
""")



In [ ]:
# LinkSourceView: ORDER_CREDITCARD_Link Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s2`.`creditcardid` AS `fk_creditcard_creditcardid`
    ,`s1`.`salesorderid` AS `fk_order_salesorderid`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderheader_result` AS `s1`
JOIN `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_creditcard_result` AS `s2`
   ON `s1`.`creditcardid` = `s2`.`creditcardid`
""")



In [ ]:
# LinkHashingView: ORDER_CREDITCARD_Link Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`fk_creditcard_creditcardid` AS String), '_NULL_') || '_' || COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `link_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`fk_creditcard_creditcardid` AS `fk_creditcard_creditcardid`
    ,SHA2(COALESCE(CAST(`fk_creditcard_creditcardid` AS String), '_NULL_'), 256) AS `creditcard_creditcard_hk`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,SHA2(COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `order_order_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard_source`
""")



In [ ]:
# LinkresultView: ORDER_CREDITCARD_Link Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`link_hk` AS `link_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`fk_creditcard_creditcardid` AS `fk_creditcard_creditcardid`
    ,`creditcard_creditcard_hk` AS `creditcard_creditcard_hk`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`order_order_hk` AS `order_order_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard`
WHERE `link_hk` <> '0'
""")



In [ ]:
# LinkDeltaView: ORDER_CREDITCARD_Link Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard_delta`
AS
SELECT
     `bg_source`.`link_hk` AS `link_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`fk_creditcard_creditcardid` AS `fk_creditcard_creditcardid`
    ,`bg_source`.`creditcard_creditcard_hk` AS `creditcard_creditcard_hk`
    ,`bg_source`.`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`bg_source`.`order_order_hk` AS `order_order_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_creditcard` AS `bg_target`
   ON `bg_source`.`link_hk` = `bg_target`.`link_hk`
WHERE `bg_target`.`link_hk` IS NULL
""")



In [ ]:
# LinkTable: ORDER_ORDERDETAIL_Link Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`link_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`fk_order_salesorderid` Integer NOT NULL
    ,`order_order_hk` String NOT NULL
    ,`fk_orderdetail_salesorderid` Integer NOT NULL
    ,`fk_orderdetail_salesorderdetailid` Integer NOT NULL
    ,`orderdetail_orderdetail_hk` String NOT NULL
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail` (
     `bg_loadtimestamp`
    ,`link_hk`
    ,`bg_sourcesystem`
    ,`fk_order_salesorderid`
    ,`order_order_hk`
    ,`fk_orderdetail_salesorderid`
    ,`fk_orderdetail_salesorderdetailid`
    ,`orderdetail_orderdetail_hk`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `link_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,0 AS `fk_order_salesorderid`
    ,'0' AS `order_order_hk`
    ,0 AS `fk_orderdetail_salesorderid`
    ,0 AS `fk_orderdetail_salesorderdetailid`
    ,'0' AS `orderdetail_orderdetail_hk`
""")



In [ ]:
# LinkSourceView: ORDER_ORDERDETAIL_Link Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,`s2`.`salesorderid` AS `fk_order_salesorderid`
    ,`s1`.`salesorderid` AS `fk_orderdetail_salesorderid`
    ,`s1`.`salesorderdetailid` AS `fk_orderdetail_salesorderdetailid`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderdetail_result` AS `s1`
JOIN `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderheader_result` AS `s2`
   ON `s1`.`salesorderid` = `s2`.`salesorderid`
""")



In [ ]:
# LinkHashingView: ORDER_ORDERDETAIL_Link Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_') || '_' || COALESCE(CAST(`fk_orderdetail_salesorderid` AS String), '_NULL_') || '_' || COALESCE(CAST(`fk_orderdetail_salesorderdetailid` AS String), '_NULL_'), 256) AS `link_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,SHA2(COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `order_order_hk`
    ,`fk_orderdetail_salesorderid` AS `fk_orderdetail_salesorderid`
    ,`fk_orderdetail_salesorderdetailid` AS `fk_orderdetail_salesorderdetailid`
    ,SHA2(COALESCE(CAST(`fk_orderdetail_salesorderid` AS String), '_NULL_') || '_' || COALESCE(CAST(`fk_orderdetail_salesorderdetailid` AS String), '_NULL_'), 256) AS `orderdetail_orderdetail_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail_source`
""")



In [ ]:
# LinkresultView: ORDER_ORDERDETAIL_Link Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`link_hk` AS `link_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`order_order_hk` AS `order_order_hk`
    ,`fk_orderdetail_salesorderid` AS `fk_orderdetail_salesorderid`
    ,`fk_orderdetail_salesorderdetailid` AS `fk_orderdetail_salesorderdetailid`
    ,`orderdetail_orderdetail_hk` AS `orderdetail_orderdetail_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail`
WHERE `link_hk` <> '0'
""")



In [ ]:
# LinkDeltaView: ORDER_ORDERDETAIL_Link Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail_delta`
AS
SELECT
     `bg_source`.`link_hk` AS `link_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`fk_order_salesorderid` AS `fk_order_salesorderid`
    ,`bg_source`.`order_order_hk` AS `order_order_hk`
    ,`bg_source`.`fk_orderdetail_salesorderid` AS `fk_orderdetail_salesorderid`
    ,`bg_source`.`fk_orderdetail_salesorderdetailid` AS `fk_orderdetail_salesorderdetailid`
    ,`bg_source`.`orderdetail_orderdetail_hk` AS `orderdetail_orderdetail_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_lnk_order_orderdetail` AS `bg_target`
   ON `bg_source`.`link_hk` = `bg_target`.`link_hk`
WHERE `bg_target`.`link_hk` IS NULL
""")



In [ ]:
# RawVaultCompositeSourceView: RVC1_SALESORDERHEADER_RawVault Composite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_rc_rvc1_salesorderheader_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_loadtimestamp`
    ,`s1`.`salesorderid` AS `salesorderid`
    ,`s2`.`currencyrateid` AS `currencyrateid`
    ,`s2`.`currencyratedate` AS `currencyratedate`
    ,`s2`.`fromcurrencycode` AS `fromcurrencycode`
    ,`s2`.`tocurrencycode` AS `tocurrencycode`
    ,`s2`.`averagerate` AS `averagerate`
    ,`s2`.`endofdayrate` AS `endofdayrate`
    ,`s2`.`modifieddate` AS `modifieddate`
    ,`s2`.`dt` AS `dt`
FROM `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_salesorderheader_result` AS `s1`
LEFT OUTER JOIN `{c3-x-db-dv#stage_files_with_crm#database_name}`.`{c3-x-db-dv#stage_files_with_crm#schema_name}`.`stg_st_currencyrate_result` AS `s2`
   ON s1.CURRENCYRATEID = s2.CURRENCYRATEID
""")



In [ ]:
# RawVaultCompositeResultView: RVC1_SALESORDERHEADER_RawVault Composite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_rc_rvc1_salesorderheader_result`
AS
SELECT
     `bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`bg_source`.`salesorderid` AS `salesorderid`
    ,`bg_source`.`currencyrateid` AS `currencyrateid`
    ,`bg_source`.`currencyratedate` AS `currencyratedate`
    ,`bg_source`.`fromcurrencycode` AS `fromcurrencycode`
    ,`bg_source`.`tocurrencycode` AS `tocurrencycode`
    ,`bg_source`.`averagerate` AS `averagerate`
    ,`bg_source`.`endofdayrate` AS `endofdayrate`
    ,`bg_source`.`modifieddate` AS `modifieddate`
    ,`bg_source`.`dt` AS `dt`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_rc_rvc1_salesorderheader_source` AS `bg_source`
""")



In [ ]:
# SatelliteTable: ORDER_CURRENCY_Satellite Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`hub_hk` String NOT NULL
    ,`bg_sourcesystem` String
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`bg_rowhash` String
    ,`currencyratedate` Timestamp
    ,`fromcurrencycode` String NOT NULL
    ,`tocurrencycode` String NOT NULL
    ,`averagerate` Decimal(19,4)
)
USING delta
"""
)
result_df = spark.sql(f"""
INSERT
INTO `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency` (
     `bg_loadtimestamp`
    ,`hub_hk`
    ,`bg_sourcesystem`
    ,`bg_validfromtimestamp`
    ,`bg_rowhash`
    ,`currencyratedate`
    ,`fromcurrencycode`
    ,`tocurrencycode`
    ,`averagerate`
)
SELECT
     '1900-01-01' AS `bg_loadtimestamp`
    ,'0' AS `hub_hk`
    ,'Unknown' AS `bg_sourcesystem`
    ,'1900-01-01' AS `bg_validfromtimestamp`
    ,'0' AS `bg_rowhash`
    ,'1900-01-01' AS `currencyratedate`
    ,'Unknown' AS `fromcurrencycode`
    ,'Unknown' AS `tocurrencycode`
    ,0 AS `averagerate`
""")



In [ ]:
# SatelliteSourceView: ORDER_CURRENCY_Satellite Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_source`
AS
SELECT
     CAST(NULL AS String) AS `bg_sourcesystem`
    ,CAST(NULL AS Timestamp) AS `bg_validfromtimestamp`
    ,`s1`.`currencyratedate` AS `currencyratedate`
    ,`s1`.`fromcurrencycode` AS `fromcurrencycode`
    ,`s1`.`tocurrencycode` AS `tocurrencycode`
    ,`s1`.`averagerate` AS `averagerate`
    ,`s1`.`salesorderid` AS `fk_order_salesorderid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_rc_rvc1_salesorderheader_result` AS `s1`
""")



In [ ]:
# SatelliteHashingView: ORDER_CURRENCY_Satellite Hashing View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_hashing`
AS
SELECT
     SHA2(COALESCE(CAST(`fk_order_salesorderid` AS String), '_NULL_'), 256) AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,SHA2(COALESCE(CAST(`currencyratedate` AS String), '_NULL_') || '_' || COALESCE(`fromcurrencycode`, '_NULL_') || '_' || COALESCE(`tocurrencycode`, '_NULL_') || '_' || COALESCE(CAST(`averagerate` AS String), '_NULL_'), 256) AS `bg_rowhash`
    ,`currencyratedate` AS `currencyratedate`
    ,`fromcurrencycode` AS `fromcurrencycode`
    ,`tocurrencycode` AS `tocurrencycode`
    ,`averagerate` AS `averagerate`
    ,`fk_order_salesorderid` AS `fk_order_salesorderid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_source`
""")



In [ ]:
# SatelliteResultView: ORDER_CURRENCY_Satellite Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`hub_hk` AS `hub_hk`
    ,`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_rowhash` AS `bg_rowhash`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `hub_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`currencyratedate` AS `currencyratedate`
    ,`fromcurrencycode` AS `fromcurrencycode`
    ,`tocurrencycode` AS `tocurrencycode`
    ,`averagerate` AS `averagerate`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency`
WHERE `hub_hk` <> '0'
""")



In [ ]:
# SatelliteDeltaView: ORDER_CURRENCY_Satellite Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_delta`
AS
SELECT
     `bg_source`.`hub_hk` AS `hub_hk`
    ,`bg_source`.`bg_sourcesystem` AS `bg_sourcesystem`
    ,`bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`bg_rowhash` AS `bg_rowhash`
    ,`bg_source`.`currencyratedate` AS `currencyratedate`
    ,`bg_source`.`fromcurrencycode` AS `fromcurrencycode`
    ,`bg_source`.`tocurrencycode` AS `tocurrencycode`
    ,`bg_source`.`averagerate` AS `averagerate`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_hashing` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_result` AS `bg_target`
   ON (`bg_source`.`hub_hk` = `bg_target`.`hub_hk`)
  AND (`bg_source`.`bg_rowhash` = `bg_target`.`bg_rowhash`)
  AND (`bg_target`.`bg_validtotimestamp` = '9999-12-31')
WHERE `bg_target`.`hub_hk` IS NULL
""")



In [ ]:
# PitTable: ORDER_PIT Table_1
spark.sql("""DROP TABLE IF EXISTS `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order`""")

spark.sql("""CREATE TABLE `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order` (
     `bg_loadtimestamp` Timestamp NOT NULL
    ,`bg_validfromtimestamp` Timestamp NOT NULL
    ,`pit_hk` String NOT NULL
    ,`order_bg_validfromtimestamp` Timestamp NOT NULL
    ,`order_order_hk` String NOT NULL
    ,`order_currency_bg_validfromtimestamp` Timestamp NOT NULL
    ,`order_currency_order_currency_hk` String NOT NULL
)
USING delta
"""
)



In [ ]:
# PitSourceView: ORDER_PIT Source View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order_source`
AS
SELECT
     `loadset`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`loadset`.`pit_hk` AS `pit_hk`
    ,COALESCE(MAX(`s1`.`bg_validfromtimestamp`), '1900-01-01') AS `order_bg_validfromtimestamp`
    ,COALESCE(MAX(`s2`.`hub_hk`), '0') AS `order_order_hk`
    ,COALESCE(MAX(`s2`.`bg_validfromtimestamp`), '1900-01-01') AS `order_currency_bg_validfromtimestamp`
    ,COALESCE(MAX(`s2`.`hub_hk`), '0') AS `order_currency_order_currency_hk`
FROM (
    SELECT
         `hub_hk` AS `pit_hk`
        ,`bg_validfromtimestamp`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order`
    UNION
    SELECT
         `hub_hk` AS `pit_hk`
        ,`bg_validfromtimestamp`
    FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency`
) AS `loadset`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_result` AS `s1`
   ON (`s1`.`hub_hk` = `loadset`.`pit_hk`)
  AND (`s1`.`bg_validfromtimestamp` <= `loadset`.`bg_validfromtimestamp`)
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order_currency_result` AS `s2`
   ON (`s2`.`hub_hk` = `loadset`.`pit_hk`)
  AND (`s2`.`bg_validfromtimestamp` <= `loadset`.`bg_validfromtimestamp`)
GROUP BY
     `loadset`.`bg_validfromtimestamp`
    ,`loadset`.`pit_hk`
""")



In [ ]:
# PitResultView: ORDER_PIT Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order_result`
AS
SELECT
     `bg_loadtimestamp` AS `bg_loadtimestamp`
    ,`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,COALESCE(LEAD(`bg_validfromtimestamp`) OVER (PARTITION BY `pit_hk` ORDER BY `bg_validfromtimestamp`), '9999-12-31') AS `bg_validtotimestamp`
    ,`pit_hk` AS `pit_hk`
    ,`order_bg_validfromtimestamp` AS `order_bg_validfromtimestamp`
    ,`order_order_hk` AS `order_order_hk`
    ,`order_currency_bg_validfromtimestamp` AS `order_currency_bg_validfromtimestamp`
    ,`order_currency_order_currency_hk` AS `order_currency_order_currency_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order`
WHERE `pit_hk` <> '0'
""")



In [ ]:
# PitDeltaView: ORDER_PIT Delta View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order_delta`
AS
SELECT
     `bg_source`.`bg_validfromtimestamp` AS `bg_validfromtimestamp`
    ,`bg_source`.`pit_hk` AS `pit_hk`
    ,`bg_source`.`order_bg_validfromtimestamp` AS `order_bg_validfromtimestamp`
    ,`bg_source`.`order_order_hk` AS `order_order_hk`
    ,`bg_source`.`order_currency_bg_validfromtimestamp` AS `order_currency_bg_validfromtimestamp`
    ,`bg_source`.`order_currency_order_currency_hk` AS `order_currency_order_currency_hk`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order_source` AS `bg_source`
LEFT OUTER JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order` AS `bg_target`
   ON (`bg_source`.`pit_hk` = `bg_target`.`pit_hk`)
  AND (`bg_source`.`bg_validfromtimestamp` = `bg_target`.`bg_validfromtimestamp`)
WHERE `bg_target`.`pit_hk` IS NULL
""")



In [ ]:
# CurrentViewResultView: ORDER_Current View Result View_1
spark.sql(f"""CREATE OR REPLACE VIEW `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_current_order`
AS
SELECT
     `s1`.`revisionnumber` AS `revisionnumber`
    ,`s1`.`orderdate` AS `orderdate`
    ,`s1`.`duedate` AS `duedate`
    ,`s1`.`shipdate` AS `shipdate`
    ,`s1`.`status` AS `status`
    ,`s1`.`onlineorderflag` AS `onlineorderflag`
    ,`s1`.`salesordernumber` AS `salesordernumber`
    ,`s1`.`purchaseordernumber` AS `purchaseordernumber`
    ,`s1`.`accountnumber` AS `accountnumber`
    ,`s1`.`customerid` AS `customerid`
    ,`s1`.`salespersonid` AS `salespersonid`
    ,`s1`.`territoryid` AS `territoryid`
    ,`s1`.`billtoaddressid` AS `billtoaddressid`
    ,`s1`.`shiptoaddressid` AS `shiptoaddressid`
    ,`s1`.`shipmethodid` AS `shipmethodid`
    ,`s1`.`creditcardid` AS `creditcardid`
    ,`s1`.`creditcardapprovalcode` AS `creditcardapprovalcode`
    ,`s1`.`subtotal` AS `subtotal`
    ,`s1`.`currencyrateid` AS `currencyrateid`
    ,`s1`.`taxamt` AS `taxamt`
    ,`s1`.`freight` AS `freight`
    ,`s1`.`totaldue` AS `totaldue`
    ,`s1`.`comment` AS `comment`
    ,`s1`.`rowguid` AS `rowguid`
    ,`s1`.`modifieddate` AS `modifieddate`
    ,`s1`.`dt` AS `dt`
    ,`s2`.`salesorderid` AS `salesorderid`
FROM `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_pit_order_result` AS `s3`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_hub_order_result` AS `s2`
   ON `s2`.`hub_hk` = `s3`.`pit_hk`
JOIN `{c3-x-db-dv#rawvault#database_name}`.`{c3-x-db-dv#rawvault#schema_name}`.`rdv_sat_order` AS `s1`
   ON (`s1`.`hub_hk` = `s3`.`order_order_hk`)
  AND (`s1`.`bg_validfromtimestamp` = `s3`.`order_bg_validfromtimestamp`)
WHERE `s3`.`bg_validtotimestamp` = '9999-12-31'
""")



In [ ]:
# RawVaultFooter



In [ ]:
# StageFooter

